In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

Generate the neutral molecule and its ions.

In [4]:
neutral = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule
ion_pos = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion with a depleted electron
ion_neg = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion with an extra electron

Do an FCI calculation on a given molecule.

In [5]:
def FCI(molecule, onv_basis):
    u_spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    u_spin_orbital_basis.lowdinOrthonormalize()
    
    u_hamiltonian = gqcpy.USQHamiltonian.Molecular(u_spin_orbital_basis, molecule)
    
    solver = gqcpy.EigenproblemSolver.Dense()
    environment = gqcpy.CIEnvironment.Dense(u_hamiltonian, onv_basis)

    ci_parameters = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
    
    return ci_parameters

Build an ONV basis for the neutral molecule and its associated ions.

In [7]:
K = 7

onv_basis_I = gqcpy.SpinResolvedONVBasis(K, 5, 4)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
onv_basis_J = gqcpy.SpinResolvedONVBasis(K, 5, 5)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons

Build FCI wave functions based on the ONV bases.

In [9]:
Psi_J = FCI(neutral, onv_basis_J)
Psi_I = FCI(ion_pos, onv_basis_I)

Construct Dyson orbitals using the "pseudo overlap" between an N-1 wave function and a N wave function with an annihilated electron in each spinor.

In [13]:
d_IJ = gqcpy.DysonOrbital.TransitionAmplitudes(Psi_J, Psi_I)
d_IJ.amplitudes()

array([ 3.87269627e-16, -2.14325442e-15, -2.17202118e-14,  3.34082558e-14,
       -9.43605291e-01,  9.04945553e-15, -1.96580681e-14])